# Multi Layer (3 hidden, 1 output) Impulse Response Recurent Network (IRRN, Pachitariu 2013) to sMNIST/rowMNIST
## This code uses flax.linen.scan to speed up the loop over time

## Equations
The recurrent weights are now diagonal matrices, which means we can do hadamard products instead of matrix multiplications. 

All weights are sampled using LeCun normal initialization.

All biases are initialized to zero.

Initialization:

$
W_{hx}^1 \in \mathbb{R}^{n_h \times n_x} \text{ with } W_{hx}^1 \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_x}}) \\
W_{hx}^2, W_{hx}^3 \in \mathbb{R}^{n_h \times n_h} \text{ with } W_{hx}^2, W_{hx}^3 \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_h}}) \\
\vec{w}_{hh}^1, \vec{w}_{hh}^2, \vec{w}_{hh}^3\in \mathbb{R}^{n_h} \text{ with } \vec{w}_{hh}^1, \vec{w}_{hh}^2, \vec{w}_{hh}^3 \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_h}}) \\
\vec{b}_h^1, \vec{b}_h^2, \vec{b}_h^3 \in \mathbb{R}^{n_h} \\
W_{hy} \in \mathbb{R}^{n_y \times n_h} \text{ with } W_{hy} \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_h}}) \\
\vec{b}_o \in \mathbb{R}^{n_y} \\
$


State update:

$
\vec{h}_t^1 = \text{ReLU}(W_{hx}^1\vec{x}_t + \vec{w}_{hh}^1\odot\vec{h}_{t-1}^1 + \vec{b}_h^1) \text{ with }\vec{h}_t^1 \in \mathbb{R}^{n_h} \text{ and } \vec{x}_t \in \mathbb{R}^{n_x} \\
\vec{h}_t^2 = \text{ReLU}(W_{hx}^2\vec{h}_t^1 + \vec{w}_{hh}^2\odot\vec{h}_{t-1}^2 + \vec{b}_h^2) \text{ with }\vec{h}_t^2 \in \mathbb{R}^{n_h} \\
\vec{h}_t^3 = \text{ReLU}(W_{hx}^3\vec{h}_t^2 + \vec{w}_{hh}^3\odot\vec{h}_{t-1}^2 + \vec{b}_h^3) \text{ with }\vec{h}_t^3 \in \mathbb{R}^{n_h} \\
$

Output: mean of output over time (or cummulative sum of output over time):

$
\vec{o}_t = W_{hy}\vec{h}_t^3 + \vec{b}_o \\
\vec{o} = \frac{1}{T}\sum_{t=1}^{T}\vec{o}_t \\
\vec{y} = \text{softmax}(\vec{o}) \\
$

<font color='red'>THINGS TO UNDERSTAND</font>
- why init in glorot using (256,) does not work? --> my own init function
- understand why getting the params in the layers doesn't work
- understand why not running on GPU
- exploding activations --> change the recurrent weight initialization to ensure |w| < 1 (here we see 1.5)

In [1]:
!pwd
!which python
!pip freeze | grep -E 'flax|jax|orbax|optax'

/home/tristan/LearningJAX/Flax
/home/tristan/miniconda3/envs/.jax_conda_env_LearningJAX/bin/python
flax==0.8.0
jax==0.4.25
jaxlib==0.4.25+cuda11.cudnn86
optax==0.1.8
orbax-checkpoint==0.5.0


In [2]:
import jax
import numpy as np
import torch
from jax import numpy as jnp
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
from flax import linen as nn
from flax.training import train_state
import optax
from jax.nn.initializers import lecun_normal
from typing import Any, Tuple, Sequence, Optional
import matplotlib.pyplot as plt
px = 1 / plt.rcParams['figure.dpi']

jnp.set_printoptions(precision=3, suppress=True)

# set cuda visible devices
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
# WARNING: this code is from QSSM project and won't be updated 
def create_mnist_classification_dataset(bsz=128, root="./data", version="sequential"):
    print("[*] Generating MNIST Classification Dataset...")
    assert version in ["sequential", "row"], "Invalid version for MNIST dataset"

    # Constants
    if version == "sequential":
        SEQ_LENGTH, N_CLASSES, IN_DIM = 784, 10, 1
    elif version == "row":
        SEQ_LENGTH, N_CLASSES, IN_DIM = 28, 10, 28
    tf = [
        transforms.ToTensor(),
        transforms.Normalize(mean=0.5, std=0.5)
    ]

    tf.append(transforms.Lambda(lambda x: x.view(SEQ_LENGTH, IN_DIM)))
    tf = transforms.Compose(tf)

    train = torchvision.datasets.MNIST(
        root, train=True, download=True, transform=tf
    )
    test = torchvision.datasets.MNIST(
        root, train=False, download=True, transform=tf
    )

    def custom_collate_fn(batch):
        transposed_data = list(zip(*batch))
        labels = np.array(transposed_data[1])
        images = np.array(transposed_data[0])

        return images, labels       


    # Return data loaders, with the provided batch size
    trainloader = torch.utils.data.DataLoader(
        train, batch_size=bsz, shuffle=True, collate_fn=custom_collate_fn, drop_last=True
    )
    testloader = torch.utils.data.DataLoader(
        test, batch_size=bsz, shuffle=False, collate_fn=custom_collate_fn, drop_last=True
    )

    return trainloader, testloader, N_CLASSES, SEQ_LENGTH, IN_DIM


In [4]:
dataset_version = "row"
trainloader, testloader, N_CLASSES, SEQ_LENGTH, IN_DIM = create_mnist_classification_dataset(root="../data", version=dataset_version)

[*] Generating MNIST Classification Dataset...


In [5]:
batch_x, batch_y = next(iter(testloader))
print(batch_x.shape, batch_y.shape)
print(batch_y.dtype)

(128, 28, 28) (128,)
int64


In [6]:
from typing import Sequence

class IRRNCell(nn.Module):
    # hidden_size: int
    # out_size: int

    def setup(self):
        # self.Linear_in = nn.Dense(self.hidden_size, name='Whx')
        self.Linear_in = nn.Dense(256, name='Whx')
        # self.wh = self.param('wh', lecun_normal(), (self.hidden_size,))
        self.wh = self.param('wh', lecun_normal(), (2,256))
        self.wh = self.wh[0]
        # self.Linear_out = nn.Dense(self.out_size, name='Woh')
        self.Linear_out = nn.Dense(10, name='Woh')


    def __call__(self, state, x):
        # Wh @ h + Wx @ x + b can be efficiently computed
        # by concatenating the vectors and then having a single dense layer
        xb = self.Linear_in(x)
        decayed_state = self.wh * state
        new_state = jax.nn.relu(xb + decayed_state)
        out = self.Linear_out(new_state)
        return new_state, (new_state, out)
    
class SimpleScan(nn.Module):
    # hidden_size: int
    # out_size: int

    @nn.compact
    def __call__(self, state, x):
        IRRN = nn.scan(
            IRRNCell,
            variable_broadcast='params',
            split_rngs={'params': False},
            in_axes=0,
            out_axes=0,
        )
        return IRRN()(state, x)
    
class RNNLayer(nn.Module):
    hidden_size: int
    out_size: int

    def setup(self):
        self.SimpleScan = SimpleScan()


    def __call__(self, x):
        state = jnp.zeros((self.hidden_size,))
        
        _, (state_hist, out_hist) = self.SimpleScan(state, x)
        return state_hist, out_hist

class RNN(nn.Module):
    hidden_size: int
    output_size: int
    n_layers: int

    def setup(self):
        self.rnn_cells = [RNNLayer(self.hidden_size, self.output_size) for _ in range(self.n_layers)]
    def __call__(self, x):
        # print(x.shape)
        all_state_hist = []
        for id_layer, rnn_cell in enumerate(self.rnn_cells[:-1]):
            x, _ = rnn_cell(x)
            all_state_hist.append(x)
        state_hist, out_hist = self.rnn_cells[-1](x)
        all_state_hist.append(state_hist)
        return all_state_hist, out_hist
    
BatchRNN = nn.vmap(RNN, in_axes=0, out_axes=0, variable_axes={'params': None}, split_rngs={'params': False})


In [7]:
srn = BatchRNN(256, 10, 3)
params = srn.init(jax.random.PRNGKey(0), jnp.zeros_like(batch_x[:5]))
all_state_hist, out_hist = srn.apply(params, batch_x[:5])
print(len(all_state_hist), all_state_hist[0].shape, all_state_hist[1].shape, all_state_hist[2].shape, out_hist.shape) # (batch, time, cell_size)
y = out_hist.mean(axis=1)
print(y.shape)
assert y.shape == (5, 10)
print(y)

3 (5, 28, 256) (5, 28, 256) (5, 28, 256) (5, 28, 10)
(5, 10)
[[-5785.811 -6685.502 -2568.396  3119.149  5653.399  -184.029  -540.825
   2908.618 -5782.951  2144.063]
 [-4867.076 -6485.928 -2908.41   3038.869  7611.862  -342.252  1384.696
   3928.536 -4474.076  3252.68 ]
 [-5802.188 -4749.925 -2123.973  2374.383  5576.978   249.23   -793.223
   2005.332 -4698.717  2229.069]
 [-5397.569 -6143.968 -2572.076  2900.378  6265.029  -570.091   247.036
   3619.686 -4825.208  2368.651]
 [-5715.976 -6652.2   -2664.757  3100.221  5810.511  -282.484  -408.916
   3110.404 -5714.659  2270.614]]


In [8]:
one_hot = jax.nn.one_hot(batch_y[:5], 10)
loss = optax.softmax_cross_entropy(logits=y, labels=one_hot)
print(loss.shape)
print(loss)
print(loss.mean())


(5,)
[ 2744.781 10520.271 10326.903 11662.599    -0.   ]
7050.911


In [9]:
# print(params['params'].keys())
# print(params['params']['RNNLayer_0'].keys())
# print(params['params']['RNNLayer_0']['ScanRNNCell_0'].keys())

print('\nparams: \n', jax.tree_util.tree_map(jnp.shape, params['params']))

# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_Out']['kernel'].shape == (256, 10)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_Out']['bias'].shape == (10,)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_RNN']['kernel'].shape == (257, 256)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_RNN']['bias'].shape == (256,)


params: 
 {'rnn_cells_0': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (28, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_1': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_2': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}}


In [10]:

t = jnp.arange(784) if dataset_version == "sequential" else jnp.arange(28)

nb_inputs_to_plot = 5
nb_components_to_plot = 5
fig, axs = plt.subplots(5, 1, figsize=(1200*px, 1200*px))
labels_input = [f'pixel_{i}' for i in range(784)] if dataset_version == "sequential" else [f'col_{i}' for i in range(nb_inputs_to_plot)]
axs[0].plot(t, batch_x[0, :, 14-nb_inputs_to_plot//2:14+(nb_inputs_to_plot+1)//2], label=labels_input)
axs[0].set_title('Input Sequence')
axs[0].legend(ncol=14, loc='upper center', bbox_to_anchor=(0.5, -0.1))
axs[1].plot(t, all_state_hist[0][0, :, :nb_components_to_plot], label=[f'state_{i}' for i in range(nb_components_to_plot)])
axs[1].set_title('State History - Layer 1')
legend_ncol = 5 if nb_components_to_plot > 5 else nb_components_to_plot
axs[1].legend(ncol=legend_ncol, loc='upper center', bbox_to_anchor=(0.5, -0.1))
axs[2].plot(t, all_state_hist[1][0, :, :nb_components_to_plot], label=[f'state_{i}' for i in range(nb_components_to_plot)])
axs[2].set_title('State History - Layer 2')
axs[2].legend(ncol=legend_ncol, loc='upper center', bbox_to_anchor=(0.5, -0.1))
axs[3].plot(t, all_state_hist[2][0, :, :nb_components_to_plot], label=[f'state_{i}' for i in range(nb_components_to_plot)])
axs[3].set_title('State History - Layer 3')
axs[-1].plot(t, out_hist[0, :, :], label=[f'out_{i}' for i in range(10)])
axs[-1].set_title('Output History')
axs[-1].legend(ncol=5, loc='upper center', bbox_to_anchor=(0.5, -0.1))
plt.tight_layout()
plt.show()

In [11]:
key = jax.random.PRNGKey(0)
key, subkey = jax.random.split(key)
lr = 1e-3

In [12]:
@jax.jit
def update_model(state, grads):
    return state.apply_gradients(grads=grads)

In [13]:
def apply_model(state, images, labels):
    """Computes gradients, loss and accuracy for a single batch."""

    def loss_fn(params):
        _, out_hist = state.apply_fn({'params': params}, images)
        logits = out_hist.mean(axis=1)
        one_hot = jax.nn.one_hot(labels, 10)
        loss = jnp.mean(optax.softmax_cross_entropy(logits=logits, labels=one_hot))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    return grads, loss, accuracy
    

In [14]:
def run_epoch(state, train_dl, rng):
    """Train for a single epoch."""

    epoch_loss = []
    epoch_accuracy = []

    progress_bar = tqdm(train_dl, desc="Training", leave=True)
    batch_id = 0
    for batch_images, batch_labels in progress_bar:
        grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
        # print(jnp.max(grads['dense_0']['kernel']), jnp.min(grads['dense_0']['kernel']))
        # print(loss)
        state = update_model(state, grads)
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)
        batch_id += 1
        if batch_id % 3 == 0:
            progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy.item())
        
    train_loss = np.mean(epoch_loss)
    train_accuracy = np.mean(epoch_accuracy)
    return state, train_loss, train_accuracy


In [15]:
def create_train_state(key, model_cls, lr, dataset_version):
    
    init_x = jnp.ones((128, 784, 1)) if dataset_version == "sequential" else jnp.ones((128, 28, 28))

    model = model_cls(hidden_size=256, output_size=10, n_layers=3)
    params = model.init(key, init_x)['params']
    
    # Debugging: Print parameter structure
    print("Initialized parameter structure:", jax.tree_util.tree_map(jnp.shape, params))

    optimizer = optax.adam(lr)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params,
        tx=optimizer,
    )


In [16]:
state = create_train_state(key, BatchRNN, lr, dataset_version)

Initialized parameter structure: {'rnn_cells_0': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (28, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_1': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_2': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}}


In [17]:
jax.devices()

[cuda(id=0)]

We can see that 1 epoch takes ~2min with this scan version.
The vanilla version takes ~2min for 10 batches --> 92 min / epoch.

In [18]:
for epoch in range(3):
    state, train_loss, train_accuracy = run_epoch(state, trainloader, key)
    print(f"Epoch {epoch} | Loss: {train_loss} | Accuracy: {train_accuracy}")

Training: 100%|██████████| 468/468 [08:49<00:00,  1.13s/it, accuracy=0.539, loss=8.28]   


Epoch 0 | Loss: 106.93313598632812 | Accuracy: 0.44107237458229065


Training: 100%|██████████| 468/468 [08:59<00:00,  1.15s/it, accuracy=0.516, loss=4.96]


Epoch 1 | Loss: 4.387658596038818 | Accuracy: 0.6135650277137756


Training: 100%|██████████| 468/468 [09:06<00:00,  1.17s/it, accuracy=0.719, loss=2.45]


Epoch 2 | Loss: 2.66204571723938 | Accuracy: 0.6843282580375671


In [27]:
all_state_hist, out_hist = srn.apply(params, batch_x[:5])
print([all_state_hist[i].shape for i in range(3)], out_hist.shape)

[(5, 28, 256), (5, 28, 256), (5, 28, 256)] (5, 28, 10)


In [29]:
print(jax.tree_util.tree_map(jnp.shape, params['params']))
print(jax.tree_util.tree_map(jnp.shape, state.params))

{'rnn_cells_0': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (28, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_1': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_2': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}}
{'rnn_cells_0': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (28, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_1': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}, 'rnn_cells_2': {'SimpleScan': {'ScanIRNNCell_0': {'Whx': {'bias': (256,), 'kernel': (256, 256)}, 'Woh': {'bias': (10,), 'kernel': (256, 10)}, 'wh': (2, 256)}}}}


In [42]:
fig, axs = plt.subplots(3, 3, figsize=(1200*px, 600*px))
axs[0,0].set_title('Whx Kernel')
axs[0,1].set_title('wh')
axs[0,2].set_title('Woh kernel')
for l in range(3):
    min_whx_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].min()
    min_whx = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].min()
    min_whx = min(min_whx_init, min_whx)

    min_wh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].min()
    min_wh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].min()
    min_wh = min(min_wh_init, min_wh)

    min_woh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].min()
    min_woh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].min()
    min_woh = min(min_woh_init, min_woh)

    max_whx_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].max()
    max_whx = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].max()
    max_whx = max(max_whx_init, max_whx)

    max_wh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].max()
    max_wh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].max()
    max_wh = max(max_wh_init, max_wh)

    max_woh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].max()
    max_woh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].max()
    max_woh = max(max_woh_init, max_woh)



    axs[l,0].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_whx, max_whx))
    axs[l,1].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].flatten(), bins=100, alpha=0.5, range=(min_wh, max_wh))
    axs[l,2].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_woh, max_woh))
    axs[l,0].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Whx']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_whx, max_whx))
    axs[l,1].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'].flatten(), bins=100, alpha=0.5, range=(min_wh, max_wh))
    axs[l,2].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['Woh']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_woh, max_woh))

    # y axis title
    axs[l,0].set_ylabel(f'L{l+1}: [{min_whx:.2f}, {max_whx:.2f}]')
    axs[l,1].set_ylabel(f'L{l+1}: [{min_wh:.2f}, {max_wh:.2f}]')
    axs[l,2].set_ylabel(f'L{l+1}: [{min_woh:.2f}, {max_woh:.2f}]')
    # set y range
plt.tight_layout()
plt.show()

In [37]:
jnp.min(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRNNCell_0']['wh'])

Array(-1.584, dtype=float32)